### Что изменено?
- Qwen 2 7B заменён на Qwen 2.5 3B (более новая модель, плюс неквантизованная модель на 7 миллиардов параметров у меня не влезает в видеопамять, а вот на 3 миллиарда более чем)
- Методом научного тыка улучшены системный и пользовательский промпты
- Попытка улучшить качество путём нормализации текста (fail)
- Расчистка кода

In [ ]:
!pip install datasets -q
!pip install transformers -q
!pip install accelerate -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q

# https://stackoverflow.com/questions/53247985/tqdm-4-28-1-in-jupyter-notebook-intprogress-not-found-please-update-jupyter-an
!pip install ipywidgets -q
!pip install jupyterlab -q

In [95]:
from typing import Dict, List, Union

import datasets
import transformers

from fuzzywuzzy import fuzz, process

import sklearn.metrics as m
from tqdm.notebook import tqdm
import numpy as np

In [96]:
MessageHint = Dict[
    str,
    Union[
        List[Dict[str, str]],
        List[List[Dict[str, str]]]
    ]
]

### 1. Функция для создания промптов к модели

Здесь задаём промпты для системы (контекст для самой модели) и пользователя (что от модели требуется).

In [97]:
prompt_system = 'Вы — очень умный и очень полезный помощник, всегда дающий ' \
                'только продуманные верные ответы, умеющий читать тексты ' \
                'на русском языке, глубоко понимать их и анализировать. '

Что интересно — промпт, который требует Qwen строго придерживаться списка тем:

In [98]:
prompt_user = 'Прочтите, пожалуйста, следующий текст и определите, ' \
              'какая ОДНА тема из предоставленного списка НАИБОЛЕЕ представлена' \
              'в следующем тексте. В качестве ответа напишите ТОЛЬКО ОДНУ тему,' \
              'наиболее подходящую, СТРОГО ИЗ СПИСКА, и больше ничего. Спасибо!'

В конечном итоге работает ХУЖЕ, чем промпт, прямо разрешающий модели отклоняться и выдумать свои категории:

In [99]:
prompt_user = 'Прочтите, пожалуйста, следующий текст и определите, ' \
              'какая ОДНА тема из списка тем ниже НАИБОЛЕЕ представлена ' \
              'в следующем тексте. В качестве ответа напишите НАИБОЛЕЕ ' \
              'подходящую тему, ОДНИМ СЛОВОМ НА АНГЛИЙСКОМ, желательно ' \
              '(но не обязательно) взятую из списка тем ниже — главное, ' \
              'чтобы у текста была СТРОГО ОПРЕДЕЛЁННАЯ тема. Спасибо! '

In [100]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> MessageHint:
    assert len(categories) >= 2, f'Ожидалось 2+ категорий, получено {len(categories)}'
 
    categories_as_string = '; '.join(categories)

    prompt_user_full = prompt_user + \
                    f'\nСПИСОК ТЕМ ЧЕРЕЗ ";": "{categories_as_string}" ' \
                    f'\nТЕКСТ: "{text}" \nВАШ ОТВЕТ: '

    messages = [
        {
            'role': 'system',
            'content': prompt_system
        },
        {
            'role': 'user',
            'content': prompt_user_full
        }
    ]

    return {'message_for_llm': messages}

In [101]:
test_message = """
Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).
Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся с
неразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенно
различными самостоятельными субстанциями и не могут, следовательно, иметь ничего общего, никаких «точек касания».
В конечном итоге дуалисты вынуждены соглашаться либо с материалистическим, либо с идеалистическим пониманием мира.
""".replace('\n', '')

test_categories = ['философия', 'математика', 'физика']

print(prepare_message_for_llm(test_message, test_categories))

{'message_for_llm': [{'role': 'system', 'content': 'Вы — очень умный и очень полезный помощник, всегда дающий только продуманные верные ответы, умеющий читать тексты на русском языке, глубоко понимать их и анализировать. '}, {'role': 'user', 'content': 'Прочтите, пожалуйста, следующий текст и определите, какая ОДНА тема из списка тем ниже НАИБОЛЕЕ представлена в следующем тексте. В качестве ответа напишите НАИБОЛЕЕ подходящую тему, ОДНИМ СЛОВОМ НА АНГЛИЙСКОМ, желательно (но не обязательно) взятую из списка тем ниже — главное, чтобы у текста была СТРОГО ОПРЕДЕЛЁННАЯ тема. Спасибо! \nСПИСОК ТЕМ ЧЕРЕЗ ";": "философия; математика; физика" \nТЕКСТ: "Крупнейшим дуалистом был великий французский философ, физик и математик Рене Декарт (1596–1650).Дуализм не оформился в самостоятельное философское направление (как материализм и идеализм), поскольку столкнулся снеразрешимой проблемой. Невозможно объяснить, как взаимодействуют материя и сознание, если они являются совершенноразличными самостоятел

### 2. Загрузим модель Qwen 2.5

Technical report — https://arxiv.org/pdf/2407.10671

https://qwenlm.github.io/blog/qwen2/

* Размеры от 0.5B до 72B.
* 29 языков, включая русский.
* Поддержка контекстного окна до 128K токенов у Qwen2-7B-Instruct и Qwen2-72B-Instruct.
* Использование Grouped Query Attention (GQA).

---

Если "никочает", читать: https://stackoverflow.com/questions/76707715/stucking-at-downloading-shards-for-loading-llm-model-from-huggingface

TLDR: Модель с HF скачивается невероятно долго, а TQDM
показывает прогресс только по окончании скачивания её куска
(или всей модели, если она маленькая и выложена одним файлов).
Через минут 5 1-й shard модели докачается и это будет видно...

In [102]:
# LLM_PIPELINE = transformers.pipeline(model='Qwen/Qwen2.5-0.5B-Instruct', device_map='cuda:0', torch_dtype='auto')
LLM_PIPELINE = transformers.pipeline(model='Qwen/Qwen2.5-3B-Instruct', device_map='cuda:0', torch_dtype='auto')
# LLM_PIPELINE = transformers.pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### З. 3агрузим текстовый корпус
https://huggingface.co/datasets/Davlan/sib200/viewer/rus_Cyrl

In [103]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train') # <-- НЕ ИСПОЛЬЗУЕТСЯ!
validation_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [104]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Категории классификации текстов: \n{list_of_categories}')

Категории классификации текстов: 
['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [105]:
print(validation_set)
print(validation_set[:1])

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})
{'index_id': [548], 'category': ['sports'], 'text': ['Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.']}


### ~~4. Нормализация текстов через Spacy~~

На практике это только ухудшает качество, я думаю это потому что изначально модель обучалась не на нормализованных текстах, а мы дообучения не выполняем, и по сути нормализацией лишь уменьшаем информативность текста.

In [ ]:
import spacy
!python3 -m spacy download ru_core_news_sm

In [107]:
NLP_PIPELINE = spacy.load('ru_core_news_sm')

In [108]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)

    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]

    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [109]:
validation_set_norm = validation_set.map(
    lambda x: {'text': normalize_text(x['text'], NLP_PIPELINE)}
)

test_set_norm = test_set.map(
    lambda x: {'text': normalize_text(x['text'], NLP_PIPELINE)}
)

In [110]:
print(validation_set[38]['text'])
print(validation_set_norm[38]['text'])

Беспилотный лунный орбитальный аппарат "Чандраян-1" выбросил свой лунный ударный зонд (MIP), который пронёсся над поверхностью Луны со скоростью 1,5 километра в секунду (3000 миль в час), и успешно совершил жесткую посадку вблизи южного полюса Луны.
беспилотный лунный орбитальный аппарат чандраян-1 выбросить свой лунный ударный зонд mip который пронестись над поверхность луна со скорость <NUM> километр в секунда <NUM> миля в час и успешно совершить жёсткий посадка вблизи южный полюс луна


### 5. Добавим prompt

In [111]:
validation_set_for_llm = validation_set.map(
    lambda it: prepare_message_for_llm(it['text'], list_of_categories)
)
test_set_for_llm = test_set.map(
    lambda it: prepare_message_for_llm(it['text'], list_of_categories)
)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [112]:
print(validation_set_for_llm)
print(validation_set_for_llm['message_for_llm'][0])

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})
[{'content': 'Вы — очень умный и очень полезный помощник, всегда дающий только продуманные верные ответы, умеющий читать тексты на русском языке, глубоко понимать их и анализировать. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая ОДНА тема из списка тем ниже НАИБОЛЕЕ представлена в следующем тексте. В качестве ответа напишите НАИБОЛЕЕ подходящую тему, ОДНИМ СЛОВОМ НА АНГЛИЙСКОМ, желательно (но не обязательно) взятую из списка тем ниже — главное, чтобы у текста была СТРОГО ОПРЕДЕЛЁННАЯ тема. Спасибо! \nСПИСОК ТЕМ ЧЕРЕЗ ";": "entertainment; geography; health; politics; science/technology; sports; travel" \nТЕКСТ: "Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость." \nВАШ ОТВЕТ: ', 'role': 'user'}]


### 6. Сгенерируем ответы модели

In [113]:
y_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=5)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))

y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

Результаты:

In [114]:
print(y_pred[0])

[{'content': 'Вы — очень умный и очень полезный помощник, всегда дающий только продуманные верные ответы, умеющий читать тексты на русском языке, глубоко понимать их и анализировать. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая ОДНА тема из списка тем ниже НАИБОЛЕЕ представлена в следующем тексте. В качестве ответа напишите НАИБОЛЕЕ подходящую тему, ОДНИМ СЛОВОМ НА АНГЛИЙСКОМ, желательно (но не обязательно) взятую из списка тем ниже — главное, чтобы у текста была СТРОГО ОПРЕДЕЛЁННАЯ тема. Спасибо! \nСПИСОК ТЕМ ЧЕРЕЗ ";": "entertainment; geography; health; politics; science/technology; sports; travel" \nТЕКСТ: "Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость." \nВАШ ОТВЕТ: ', 'role': 'user'}, {'role': 'assistant', 'content': 'health'}]


In [115]:
cat = [x[-1]['content'].lower() for x in y_pred]
unique, counts = np.unique(cat, return_counts=True)

for i in range(len(unique)):
    print(unique[i], ':', counts[i])

entertainment : 9
geography : 22
health : 12
politics : 19
science/technology : 19
sleep : 1
sports : 4
travel : 13


In [116]:
print(m.classification_report(
    y_true=y_true,
    y_pred=[x[-1]['content'].lower() for x in y_pred]
))

                    precision    recall  f1-score   support

     entertainment       0.56      0.56      0.56         9
         geography       0.36      1.00      0.53         8
            health       0.58      0.64      0.61        11
          politics       0.68      0.93      0.79        14
science/technology       0.95      0.72      0.82        25
             sleep       0.00      0.00      0.00         0
            sports       1.00      0.33      0.50        12
            travel       0.62      0.40      0.48        20

          accuracy                           0.64        99
         macro avg       0.59      0.57      0.54        99
      weighted avg       0.73      0.64      0.64        99



In [117]:
print(
    'Категорий в датасете после прогона моделью:',
    len(set([x[-1]['content'].lower() for x in y_pred])),
    set([x[-1]['content'].lower() for x in y_pred])
)
print(
    '\nКатегорий в датасете на самом деле:',
    len(set(list_of_categories)),
    set(list_of_categories)
)
print(
    '\nКатегорий выдумала модель:',
    len(set([x[-1]['content'].lower() for x in y_pred]) - set(list_of_categories)),
    set([x[-1]['content'].lower() for x in y_pred]) - set(list_of_categories)
)

Категорий в датасете после прогона моделью: 8 {'travel', 'science/technology', 'sports', 'geography', 'health', 'sleep', 'politics', 'entertainment'}

Категорий в датасете на самом деле: 7 {'travel', 'science/technology', 'sports', 'geography', 'health', 'politics', 'entertainment'}

Категорий выдумала модель: 1 {'sleep'}


**Модель сгенерировала лишние ответы?**

### 7. Применим постобработку ответов модели

Воспользуемся расстоянием Левенштейна

In [118]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(
        it[-1]['content'].lower(),
        list_of_categories,
        scorer=fuzz.token_sort_ratio
    )[0],
    tqdm(y_pred)
))

  0%|          | 0/99 [00:00<?, ?it/s]

In [119]:
print(m.classification_report(
    y_true=y_true,
    y_pred=y_pred_with_normalization
))

                    precision    recall  f1-score   support

     entertainment       0.56      0.56      0.56         9
         geography       0.36      1.00      0.53         8
            health       0.58      0.64      0.61        11
          politics       0.68      0.93      0.79        14
science/technology       0.95      0.72      0.82        25
            sports       0.80      0.33      0.47        12
            travel       0.62      0.40      0.48        20

          accuracy                           0.64        99
         macro avg       0.65      0.65      0.61        99
      weighted avg       0.70      0.64      0.63        99



### 8. Посмотрим результаты на тестовом подмножестве

Вот без постобработки:

In [120]:
y_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=5)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [121]:
print(m.classification_report(
    y_true=y_true,
    y_pred=[x[-1]['content'].lower() for x in y_pred]
))

                          precision    recall  f1-score   support

                 biology       0.00      0.00      0.00         0
           entertainment       0.79      0.58      0.67        19
               geography       0.41      0.88      0.56        17
                  health       0.73      0.86      0.79        22
              literature       0.00      0.00      0.00         0
                    none       0.00      0.00      0.00         0
none of the given topics       0.00      0.00      0.00         0
                politics       0.72      0.97      0.83        30
      science/technology       0.91      0.61      0.73        51
                  sports       0.89      0.68      0.77        25
          transportation       0.00      0.00      0.00         0
                  travel       0.79      0.57      0.67        40

                accuracy                           0.71       204
               macro avg       0.44      0.43      0.42       204
        

А вот — после применения постобработки:

In [122]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(
        it[-1]['content'].lower(),
        list_of_categories,
        scorer=fuzz.token_sort_ratio
    )[0],
    y_pred
))

In [123]:
print(m.classification_report(
    y_true=y_true,
    y_pred=y_pred_with_normalization
))

                    precision    recall  f1-score   support

     entertainment       0.80      0.63      0.71        19
         geography       0.41      0.88      0.56        17
            health       0.73      0.86      0.79        22
          politics       0.72      0.97      0.83        30
science/technology       0.92      0.65      0.76        51
            sports       0.85      0.68      0.76        25
            travel       0.77      0.57      0.66        40

          accuracy                           0.73       204
         macro avg       0.74      0.75      0.72       204
      weighted avg       0.78      0.73      0.73       204



Для сравнения: ноутбук без изменений, но с этой же моделью, даёт результат f1-score = 0.67.

### "Уходя, гасите свет"

In [124]:
UNLOAD_MODEL = True

if UNLOAD_MODEL:
    try:
        del LLM_PIPELINE
    except:
        pass

    import torch
    torch.cuda.empty_cache()
    import gc
    gc.collect()